In [28]:
import numpy as np
seed = 123
np.random.seed(seed)
import collections, copy, pickle
from importlib import reload
from dateutil.parser import parse
import scipy.linalg, scipy.stats
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 14
# rcParams['text.usetex'] = True
import seaborn as sns
from IPython.display import HTML
import scipy.stats
from sklearn import preprocessing
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [29]:
import util.data
import util.ndcg
# import util.plot

In [30]:
data = pd.read_csv('data/training_set_VU_DM_clean.csv', sep=';', nrows=100000)
data_test = pd.read_csv('data/test_set_VU_DM.csv', sep=',', nrows=100000)

In [31]:
srchIdList = data['srch_id'].unique()
trainID, testID = sklearn.model_selection.train_test_split(srchIdList, random_state=42, test_size=0.4)
trainset = data[data['srch_id'].isin(trainID)]
testset = data[data['srch_id'].isin(testID)]

y_labels = ['click_bool', 'booking_bool', 'score']

def split_xy(data: pd.DataFrame):
    return data.drop(columns=y_labels), data['score']

X_train, y_train = split_xy(trainset)
# X_val, y_val = split_xy(valset)
X_test, y_test = split_xy(testset)

X_train = X_train.drop('position', 1)
X_test = X_test.drop('position', 1)

In [32]:
# use Gradient Boosting to make model of data to predict the 'score'
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1, 
                          max_depth = 5, alpha = 10, n_estimators = 10)

# fit training set
xg_reg.fit(X_train,y_train)

# fit model to test set
y_pred = xg_reg.predict(X_test)
# X_test['score'] = preds

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [33]:
# calculate dcg of test set per srch_id
Xy_pred = util.data.Xy_pred(X_test, y_pred)

# put true y values on indexes, do not sort !
Xy_pred['score'] = y_test

# calculate ideal dcg of test set per srch_id
Xy_true = util.data.Xy_pred(X_test, y_test)

# calculate NDCG
ndcg = util.ndcg.ndcg(Xy_pred, Xy_true)
ndcg

0.6893786748532622